In [ ]:
import numpy as np
import cv2 as cv
from matplotlib import pyplot as plt
import binascii
plt.rcParams["figure.figsize"] = (20,12)


def encode_as_binary_array(msg):
    """Encode a message as a binary string."""
    msg = msg.encode("utf-8")
    msg = msg.hex()
    msg = [msg[i:i + 2] for i in range(0, len(msg), 2)]
    msg = [ "{:08b}".format(int(el, base=16)) for el in msg]
    return "".join(msg)

def decode_from_binary_array(array):
    """Decode a binary string to utf8."""
    array = [array[i:i+8] for i in range(0, len(array), 8)]
    if len(array[-1]) != 8:
        array[-1] = array[-1] + "0" * (8 - len(array[-1]))
    array = [ "{:02x}".format(int(el, 2)) for el in array]
    array = "".join(array)
    result = binascii.unhexlify(array)
    return result.decode("utf-8", errors="replace")

def hide_message(image, message, nbits=1):
    """Hide a message in an image (LSB).

    nbits: number of least significant bits
    """
    nbits = clamp(nbits, 1, 8)
    shape = image.shape
    image = np.copy(image).flatten()
    if len(message) > len(image) * nbits:
        raise ValueError("Message is to long :(")

    chunks = [message[i:i + nbits] for i in range(0, len(message), nbits)]
    for i, chunk in enumerate(chunks):
        byte = "{:08b}".format(image[i])
        new_byte = byte[:-nbits] + chunk
        image[i] = int(new_byte, 2)

    return image.reshape(shape)

def clamp(n, minn, maxn):
    """Clamp the n value to be in range (minn, maxn)."""
    return max(min(maxn, n), minn)

# Zadanie 1

In [ ]:
image = cv.imread('img/trashcan.jpg')
image = cv.cvtColor(image, cv.COLOR_BGR2RGB)

message = encode_as_binary_array('SUS AMOGUS')

image_msg = hide_message(image, message, nbits=1)
plt.imshow(image_msg)

# Zadanie 2

In [ ]:
target_message_length = int(0.75 * image.size)
message_part = "What the fuck did you just fucking say about me, you little bitch? I'll have you know I graduated top of my class in the Navy Seals, and I've been involved in numerous secret raids on Al-Quaeda, and I have over 300 confirmed kills. I am trained in gorilla warfare and I'm the top sniper in the entire US armed forces. You are nothing to me but just another target. I will wipe you the fuck out with precision the likes of which has never been seen before on this Earth, mark my fucking words. You think you can get away with saying that shit to me over the Internet? Think again, fucker. As we speak I am contacting my secret network of spies across the USA and your IP is being traced right now so you better prepare for the storm, maggot. The storm that wipes out the pathetic little thing you call your life. You're fucking dead, kid. I can be anywhere, anytime, and I can kill you in over seven hundred ways, and that's just with my bare hands. Not only am I extensively trained in unarmed combat, but I have access to the entire arsenal of the United States Marine Corps and I will use it to its full extent to wipe your miserable ass off the face of the continent, you little shit. If only you could have known what unholy retribution your little \"clever\" comment was about to bring down upon you, maybe you would have held your fucking tongue. But you couldn't, you didn't, and now you're paying the price, you goddamn idiot. I will shit fury all over you and you will drown in it. You're fucking dead, kiddo.\n"
message_part = encode_as_binary_array(message_part)
message = message_part * (target_message_length // len(message_part))

images_with_secret = [hide_message(image, message, i + 1) for i, image in enumerate([image] * 8)]

_, plots = plt.subplots(2,4)
for i in range(plots.size):
    plots[i//4, i%4].imshow(images_with_secret[i])
    plots[i//4, i%4].set_title(f'nbits = {i+1}')

In [ ]:
# mean square error
plt.rcParams["figure.figsize"] = (10,6)

mean_square_errors = [np.square(image_secret.astype(np.int64) - image.astype(np.int64)).mean() for image_secret in images_with_secret]
plt.title('Mean Square Error per nbits')
plt.xlabel('nbits')
plt.ylabel('MSE')
plt.plot(range(1, 9), mean_square_errors)

# Zadanie 3

In [ ]:
import math


def hide_message(image, message, nbits=1, spos=0):
    """Hide a message in an image (LSB).

    nbits: number of least significant bits
    spos: starting position
    """
    nbits = clamp(nbits, 1, 8)
    shape = image.shape
    image = np.copy(image).flatten()
    if len(message) > len(image) * nbits:
        raise ValueError("Message is to long :(")

    chunks = [message[i:i + nbits] for i in range(0, len(message), nbits)]
    for i, chunk in enumerate(chunks):
        byte = "{:08b}".format(image[i+spos])
        new_byte = byte[:-nbits] + chunk
        image[i+spos] = int(new_byte, 2)

    return image.reshape(shape)


def reveal_message(image, nbits=1, length=0, spos=0):
    """Reveal the hidden message.

    nbits: number of least significant bits
    length: length of the message in bits.
    spos: starting position
    """
    nbits = clamp(nbits, 1, 8)
    shape = image.shape
    image = np.copy(image).flatten()
    length_in_pixels = math.ceil(length/nbits)
    if len(image) < length_in_pixels or length_in_pixels <= 0:
        length_in_pixels = len(image)

    message = ""
    i = 0
    while i < length_in_pixels:
        byte = "{:08b}".format(image[i+spos])
        message += byte[-nbits:]
        i += 1

    mod = length % -nbits
    if mod != 0:
        message = message[:mod]
    return message


message = encode_as_binary_array('message with offset')

hidden_with_offset = hide_message(image, message, nbits=1, spos=2137)
revealed_with_offset = reveal_message(hidden_with_offset, nbits=1, length=len(message), spos=2137)

assert message == revealed_with_offset

# Zadanie 4

In [ ]:
def hide_image(image, secret_image_path, nbits=1) -> (np.ndarray, int):
    with open(secret_image_path, "rb") as file:
        secret_img = file.read()

    secret_img = secret_img.hex()
    secret_img = [secret_img[i:i + 2] for i in range(0, len(secret_img), 2)]
    secret_img = ["{:08b}".format(int(el, base=16)) for el in secret_img]
    secret_img = "".join(secret_img)
    return hide_message(image, secret_img, nbits), len(secret_img)


def reveal_image(outerimage: np.ndarray, length: int, nbits: int=1) -> np.ndarray:
    """Reveal an inner image hidden in the outer image.

    outerimage: an image in which the inner image is hidden
    length: length of the inner image as a bit array
    nbits: how many bits of inner image per pixel of outerimage
    """
    assert 1 <= nbits <= 8, "The number of bits per pixel has to be between 1 to 8 inclusive"
    length_in_pixels = math.ceil(length / nbits)
    outer_pixels = outerimage.size
    assert length_in_pixels <= outer_pixels, "Message can't be longer than outer image itself"

    secret_img_pixels = outerimage.flatten()[:length_in_pixels]
    secret_img_bits = [pixel & (1 << nbits) - 1 for pixel in secret_img_pixels]
    secret_img_bitstring = ''.join([f'{pixel:08b}'[-nbits:] for pixel in secret_img_bits])
    secret_img_bytes = np.array([int(secret_img_bitstring[i:i+8], 2) for i in range(0, len(secret_img_bitstring), 8)], dtype=np.uint8)

    return cv.imdecode(secret_img_bytes, cv.IMREAD_COLOR)

image_with_secret_image, secret_length = hide_image(image, 'img/amogus.jpg', nbits=3)

secret_image = reveal_image(image_with_secret_image, secret_length, 3)

_, plots = plt.subplots(1,2)

plots[0].set_title('Original image')
plots[0].imshow(image)
plots[1].set_title('Secret image')

plots[1].imshow(cv.cvtColor(secret_image, cv.COLOR_BGR2RGB))

# Zadanie 5

In [ ]:
def reveal_image(outerimage: np.ndarray, nbits: int=1) -> np.ndarray:
    """Reveal an inner image hidden in the outer image.

    outerimage: an image in which the inner image is hidden
    nbits: how many bits of inner image per pixel of outerimage
    """
    assert 1 <= nbits <= 8, "The number of bits per pixel has to be between 1 to 8 inclusive"
    secret_img_pixels = []
    outerimage = outerimage.flatten()
    for i in range(len(outerimage)):
        if np.array_equal(outerimage[i:i+1], [0xff, 0xd9]):
            break

        secret_img_pixels.append(outerimage[i])

    secret_img_bits = [pixel & (1 << nbits) - 1 for pixel in secret_img_pixels]
    secret_img_bitstring = ''.join([f'{pixel:08b}'[-nbits:] for pixel in secret_img_bits])
    secret_img_bytes = [int(secret_img_bitstring[i:i+8], 2) for i in range(0, len(secret_img_bitstring), 8)]
    secret_img_bytes = np.array(secret_img_bytes, dtype=np.uint8)

    return cv.imdecode(secret_img_bytes, cv.IMREAD_COLOR)

secret_image_nolen = reveal_image(image_with_secret_image, 3)
assert np.array_equal(secret_image, secret_image_nolen)

plt.imshow(cv.cvtColor(secret_image_nolen, cv.COLOR_BGR2RGB))